# 生成式问答模型

数据集中每一行为一个数据样本，json 格式。
其中，"context" 代表参考文章，"question"代表问题，"answer" 代表问题答案，"id"表示数据的index，从0开始

{"context": "违规分为:一般违规扣分、严重违规扣分、出售假冒商品违规扣分,淘宝网每年12月31日24:00点会对符合条件的扣分做清零处理,详情如下:|温馨提醒:由于出售假冒商品24≤N<48分,当年的24分不清零,所以会存在第一年和第二年的不同计分情况。", "answer": "12月31日24:00", "question": "淘宝扣分什么时候清零", "id": 203}

json文件中，同一个context和question可能有不同的answer，不同的answer属于不同的数据样本

## 1.构建数据集

In [ ]:
from torch.utils.data import Dataset
import json

In [ ]:
# 数据集加载函数，继承自Dataset类
class DuReaderQG(Dataset):
    def __init__(self, datafile):
        self.data = self.load_data(datafile)

    def load_data(self, datafile):
        data = []
        with open(datafile, 'r', encoding='utf-8') as f:
            for line in f:
                json_data = json.loads(line)
                data.append(json_data)
        return data

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        return self.data[index]

In [ ]:
train_data = DuReaderQG("./DuReaderQG/train.json")
valid_data = DuReaderQG("./DuReaderQG/dev.json")
print(f'train set size: {len(train_data)}')
print(f'valid set size: {len(valid_data)}')
print(next(iter(train_data)))

## 2.加载分词器和模型

使用DataLoader加载数据，设计批处理函数，将文本转换为模型可以接受的 token IDs，并且构建对应的目标。

加载mengzi-t5-base的分词器，使用T5Tokenizer加载分词器

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [ ]:
# 使用transformers的T5Tokenizer加载分词器
# 这个分词器需要sentencepiece库，没有的话需要pip安装下
tokenizer = T5Tokenizer.from_pretrained("./model/mengzi-t5-base")
model = T5ForConditionalGeneration.from_pretrained("./model/mengzi-t5-base")

模型文件中有一个config.json文件，查看模型的配置：
```
{
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "gradient_checkpointing": false,
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "torch_dtype": "float32",
  "transformers_version": "4.9.2",
  "use_cache": true,
  "vocab_size": 32128
}
```

**测试一下分词器和模型的生成功能**

但发现有点问题，如果input_text中有<extra_id_0>（比如）"中国的首都位于<extra_id_0>。"，就会正常生成答案。但是如果输入文本中没有<extra_id_0>这一token，结果就是空的。

这样看的话<extra_id_0>有点类似于占位符的感觉，或者提示模型有这个token才是句子结尾，如果提问中没有这个token，按照概率自然就会输出这个token（ids是32127），解码出来就是<extra_id_0>。

In [ ]:
# 输入文本
input_text = "中国的首都位于<extra_id_0>。"
# 对输入文本进行编码
input_ids = tokenizer(input_text, return_tensors="pt").input_ids
print(f"输入: {input_text}")
print("input_ids: ", input_ids)
# 生成文本
outputs = model.generate(
    input_ids,
    max_length=50,  # 生成文本的最大长度
    num_beams=5,    # Beam Search 的 beam 数量
    no_repeat_ngram_size=2,  # 避免重复的 n-gram
    early_stopping=True,     # 提前停止生成
)

# 解码生成的文本
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
# 打印结果
print(f"生成: {generated_text}")
print("输出的ids: ",outputs)

# 不跳过特殊token
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=False)
# 打印结果
print("不跳过特殊token")
print(f"生成: {generated_text}")

如果输入文本中没有<extra_id_0>这一token，输出ids就只有32127，解码是<extra_id_0>。

In [ ]:
# 输入文本
input_text = "中国的首都位于。"
# 对输入文本进行编码
input_ids = tokenizer(input_text, return_tensors="pt").input_ids
print(f"输入: {input_text}")
print("input_ids: ", input_ids)
# 生成文本
outputs = model.generate(
    input_ids,
    max_length=50,  # 生成文本的最大长度
    num_beams=5,    # Beam Search 的 beam 数量
    no_repeat_ngram_size=2,  # 避免重复的 n-gram
    early_stopping=True,     # 提前停止生成
)

# 解码生成的文本
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=False)
# 打印结果
print(f"生成: {generated_text}")
print("输出的ids: ",outputs)

## 3.数据预处理

生成式问答任务中，输入是context和question，标记是answer，这些都是文本，考虑将context和question拼接在一起作为输入，但可能会面临着文本长度超限，所以需要有个文本处理。

对于超长文本的处理，要么直接截取文本要么就是分块处理，先考虑简单的截取这一措施，整个流程跑通之后再试试分块。

**上下文分快处理（chunk）**
输入文本拼接，仿照抽取式问答任务，将问题和上下文编码为下面的形式：question </s> context </s>。
由于问题与上下文拼接后的 token 序列可能超过模型的最大输入长度，因此我们可以将上下文切分为短文本块 (chunk) 来处理，同时为了避免答案被截断，我们使用滑窗使得切分出的文本块之间有重叠。



**简单的拼接**

将上下文拼接后，简单的限制最大文本长度

In [ ]:
import torch
from torch.utils.data import DataLoader
from transformers import AutoModelForSeq2SeqLM

In [ ]:
# 设置最大输入长度和最大答案文本长度
max_input_length = 512
max_target_length = 64

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

model = T5ForConditionalGeneration.from_pretrained("./model/mengzi-t5-base")
model = model.to(device)

**批数据处理**

使用分词器提供的`as_target_tokenizer()`函数来并行地对输入和标签进行分词，

并且将标签序列中填充的 pad 字符设置为 -100 以便在计算交叉熵损失时忽略它们，

通过模型自带的`prepare_decoder_input_ids_from_labels`函数对标签进行移位操作以准备好 decoder input IDs。

In [ ]:
# 定义批处理函数
def collote_fn(batch_samples):
    batch_inputs, batch_question, batch_targets = [], [], []
    for sample in batch_samples:
        batch_inputs.append(sample['context'])
        batch_question.append(sample['question'])
        batch_targets.append(sample['answer'])
    batch_data = tokenizer(
        batch_inputs,
        batch_question,
        padding=True, 
        max_length=max_input_length,
        truncation=True, 
        return_tensors="pt"
    )
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            batch_targets, 
            padding=True, 
            max_length=max_target_length,
            truncation=True, 
            return_tensors="pt"
        )["input_ids"]
        batch_data['decoder_input_ids'] = model.prepare_decoder_input_ids_from_labels(labels)
        end_token_index = torch.where(labels == tokenizer.eos_token_id)[1]
        for idx, end_idx in enumerate(end_token_index):
            labels[idx][end_idx+1:] = -100
        batch_data['labels'] = labels
    return batch_data

In [ ]:
train_dataloader = DataLoader(train_data, batch_size=4, shuffle=True, collate_fn=collote_fn)
valid_dataloader = DataLoader(valid_data, batch_size=4, shuffle=False, collate_fn=collote_fn)

输出一个batch数据查看编码结果：`dict_keys(['input_ids', 'attention_mask', 'decoder_input_ids', 'labels'])`

In [ ]:
batch = next(iter(train_dataloader))
print(batch.keys())
print('batch shape:', {k: v.shape for k, v in batch.items()})
print('input_ids:',batch['input_ids'])
print('decoder_input_ids:',batch['decoder_input_ids'])
print('labels:',batch['labels'])
# 解码这些label看看是不是正确的
import numpy as np
label_tokens = batch["labels"].numpy()
label_tokens = np.where(label_tokens != -100, label_tokens, tokenizer.pad_token_id)
decoded_labels = tokenizer.batch_decode(label_tokens, skip_special_tokens=True)
# generated_text = tokenizer.decode(label, skip_special_tokens=True)
decoded_labels

## 4.训练模型

本文使用 T5ForConditionalGeneration 构建模型，使用 T5ForConditionalGeneration 构造的模型已经封装好了对应的损失函数，并且计算出的损失会直接包含在模型的输出 outputs 中，可以直接通过 outputs.loss 获得。

In [ ]:
from tqdm.auto import tqdm

**训练函数**

In [ ]:
# 定义训练函数，接口：dataloader，model，optimizer，lr_scheduler，
def train_loop(dataloader, model, optimizer, lr_scheduler, epoch, total_loss):
    progress_bar = tqdm(range(len(dataloader)))
    progress_bar.set_description(f'loss: {0:>7f}')
    finish_batch_num = (epoch-1) * len(dataloader)
    
    model.train()
    for batch, batch_data in enumerate(dataloader, start=1):
        batch_data = batch_data.to(device)
        outputs = model(**batch_data)
        # 内置的loss
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()

        total_loss += loss.item()
        progress_bar.set_description(f'loss: {total_loss/(finish_batch_num + batch):>7f}')
        progress_bar.update(1)
    return total_loss

**验证和测试**

在测试和验证函数中，通过 model.generate() 函数生成回答，然后计算生成回答和正确回答之间的差异，评价指标使用BLEU-1，BLEU-2，BLEU-3，BLEU-4.


In [ ]:
from evaluate import load

In [ ]:
# 加载 BLEU 指标
bleu_metric = load("bleu")

# 示例数据
predictions = ["The cat is on the mat"]  # 模型生成的文本
references = ["The cat is not on the mat"]  # 参考文本（可以有多个参考）
# 计算 BLEU 指标
results = bleu_metric.compute(predictions=predictions, references=references)
print(results)
TOKENIZE_CHINESE = lambda x: ' '.join(x)
# 示例数据
predictions = [TOKENIZE_CHINESE("今天天气还不错")]  # 模型生成的文本
references = [TOKENIZE_CHINESE("今天天气很好")]  # 参考文本（可以有多个参考）
# 计算 BLEU 指标
results = bleu_metric.compute(predictions=predictions, references=references)
print(results)

In [ ]:
def compute_metrics(preds, labels):
    # preds,labels:list[]
    b1, b2, b3, b4 = [], [], [], []
    for pred, label in zip(preds, labels):
        pred = [pred]
        references = [label]
        # 计算 BLEU 指标
        results = bleu_metric.compute(predictions=pred, references=references)
        b1.append(results["precisions"][0])
        b2.append(results["precisions"][1])
        b3.append(results["precisions"][2])
        b4.append(results["precisions"][3])
    return {
        "bleu-1": sum(b1) / len(b1),
        "bleu-2": sum(b2) / len(b2),
        "bleu-3": sum(b3) / len(b3),
        "bleu-4": sum(b4) / len(b4),}


In [ ]:
predictions = [TOKENIZE_CHINESE("今天天气还不错"), "The cat is on the mat"]
references = [TOKENIZE_CHINESE("今天天气很好"), "The cat is not on the mat"]
results = compute_metrics(predictions,references)
print(results)

In [ ]:
import numpy as np

In [ ]:
def test_loop(dataloader, model):
    preds, labels = [], []
    
    model.eval()
    for batch_data in tqdm(dataloader):
        batch_data = batch_data.to(device)
        with torch.no_grad():
            generated_tokens = model.generate(
                batch_data["input_ids"],
                attention_mask=batch_data["attention_mask"],
                max_length=max_target_length,
            ).cpu().numpy()
        if isinstance(generated_tokens, tuple):
            generated_tokens = generated_tokens[0]
        label_tokens = batch_data["labels"].cpu().numpy()

        decoded_preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
        label_tokens = np.where(label_tokens != -100, label_tokens, tokenizer.pad_token_id)
        decoded_labels = tokenizer.batch_decode(label_tokens, skip_special_tokens=True)

        preds += [' '.join(pred.strip()) for pred in decoded_preds]
        labels += [' '.join(label.strip()) for label in decoded_labels]

    # 计算bleu指标,返回bleu1-4四个指标
    result = compute_metrics(preds, labels)
    print(f"Bleu-1: {result['Bleu-1']:>0.2f} Blue-2: {result['Bleu-2']:>0.2f} Bleu-3: {result['Bleu-3']:>0.2f} Bleu-4: {result['Bleu-4']:>0.2f}\n")
    return result

In [ ]:
from transformers import AdamW, get_scheduler

learning_rate = 2e-5
epoch_num = 10

optimizer = AdamW(model.parameters(), lr=learning_rate)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=epoch_num*len(train_dataloader),
)

total_loss = 0.
best_bleu = 0.
for t in range(epoch_num):
    print(f"Epoch {t+1}/{epoch_num}\n-------------------------------")
    total_loss = train_loop(train_dataloader, model, optimizer, lr_scheduler, t+1, total_loss)
    bleu_score = test_loop(valid_dataloader, model)
    print(bleu_score)
    bleu = bleu_score['Bleu-4']
    if bleu > best_bleu:
        best_bleu = bleu
        print('saving new weights...\n')
        torch.save(model.state_dict(), f'epoch_{t+1}_valid_bleu-4_{bleu:0.4f}_model_weights.bin')
print("Done!")

测试模型

In [ ]:
article_text = """
受众在哪里，媒体就应该在哪里，媒体的体制、内容、技术就应该向哪里转变。
媒体融合关键是以人为本，即满足大众的信息需求，为受众提供更优质的服务。
这就要求媒体在融合发展的过程中，既注重技术创新，又注重用户体验。
"""

input_ids = tokenizer(
    article_text,
    return_tensors="pt",
    truncation=True,
    max_length=512
)
generated_tokens = model.generate(
    input_ids["input_ids"],
    attention_mask=input_ids["attention_mask"],
    max_length=32,
    no_repeat_ngram_size=2,
    num_beams=4
)
summary = tokenizer.decode(
    generated_tokens[0],
    skip_special_tokens=True,
    clean_up_tokenization_spaces=False
)
print(summary)

In [ ]:
generated_tokens = model.generate(
    batch["input_ids"],
    attention_mask=batch["attention_mask"],
    max_length=max_target_length
)
if isinstance(generated_tokens, tuple):
    generated_tokens = generated_tokens[0]
label_tokens = batch["labels"].cpu().numpy()

decoded_preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
label_tokens = np.where(label_tokens != -100, label_tokens, tokenizer.pad_token_id)
decoded_labels = tokenizer.batch_decode(label_tokens, skip_special_tokens=True)

preds = [' '.join(pred.strip()) for pred in decoded_preds]
labels = [' '.join(label.strip()) for label in decoded_labels]
print(preds)
print(labels)